In [1]:
import pandas as pd

import pickle

import numpy as np

import re
import string

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('white')

from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist

import operator

In [2]:
with open ('../models/05_kojak_demo.pkl', 'rb') as picklefile:
    df_loc, df_doc_topics, nmf_topics = pickle.load(picklefile)

In [3]:
neighborhoods = df_loc['neighborhood'].apply(lambda u: u.encode('utf-8')).tolist()
cities = df_loc['city'].tolist()
topics = df_loc['topic'].tolist()

zipped = zip(neighborhoods, cities, topics)

In [33]:
df_loc.head(1)

,neighborhood,city,topic
0,Adams Morgan,washingtondc,10_attractions


In [ ]:
df_loc['neighborhood']

In [45]:
neighborhood_name='Pilsen'
neighborhood_index = df_loc[df_loc['neighborhood']==neighborhood_name].index[0]
neighborhood_dist = np.array(df_doc_topics.iloc[neighborhood_index])

In [49]:
neighborhood_dist

array([[ 0.06977276,  0.16303613,  0.        ,  0.        ,  0.0292681 ,
         0.        ,  0.02358973,  0.01133142,  0.13041557,  0.18408705,
         0.        ,  0.01572815,  0.00821721,  0.        ,  0.        ,
         0.        ]])

In [4]:
test_list = [10.0, 0.0, 0.0, 0.0, \
       0.0, 0.0, 0.0, 0.0, \
       0.0, 0.0, 0.0, 0.0, \
       0.0, 0.0, 0.0, 0.0]

In [5]:
test_array = np.array(test_list)

In [6]:
nmf_topics[test_list.index(max(test_list))]

'1_taco'

In [7]:
test_distribution = test_array/sum(test_array)

In [8]:
sum(test_distribution)

1.0

In [58]:
def find_comps(df_doc_topics, test_distribution, city, num_comps=5):
    dists = {}
    for i in range(len(df_doc_topics)):
        dists[i] = cosine(df_doc_topics.ix[i].T, np.array(test_distribution))
    max_keys = sorted(dists, key=dists.get)#[:num_comps]
    comps = []
    count = 0
    #print max_keys
    #return dists
    for key in max_keys:
        if count <= 5:
            if zipped[key][1] == city:
                comps.append(zipped[key])
                count += 1
    return comps
    #print float(nmf_topics.index(zipped[max_keys[0]][2]))
    #print zipped[max_keys[0]][2]
    #print nmf_topics.index(max_keys[0])
    #return dists

In [60]:
comps = find_comps(df_doc_topics, pilsen, 'newyorkcity', num_comps=100)

In [62]:
comps[0][2]

'8_art'

In [10]:
find_comps(df_doc_topics, test_distribution)

[470, 236, 344, 48, 217]
('Melrose', 'nashville', '1_taco')
('Eastwood', 'nashville', '1_taco')
('Historic Waverly Place', 'nashville', '1_taco')
('Bellmont Hillsboro', 'nashville', '1_taco')
('East End', 'nashville', '1_taco')
0.0
1_taco


In [11]:
df_doc_topics.iloc[3]

0     0.000000
1     0.000000
2     0.170987
3     0.027954
4     0.000000
5     0.018020
6     0.004866
7     0.029608
8     0.014814
9     0.000000
10    0.000000
11    0.000000
12    0.029614
13    0.000000
14    0.000000
15    0.006484
Name: 3, dtype: float64

In [12]:
df_doc_topics.sort_values(by=0, ascending=False).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
48,0.213651,0.000000,0.017951,0.009858,0.026662,0.063114,0.000000,0.000000,0.004009,0.000000,0.004336,0.000000,0.0,0.001202,0.0,0.000523
236,0.209811,0.000000,0.003849,0.000000,0.019200,0.000000,0.017186,0.009708,0.000000,0.000000,0.022532,0.000000,0.0,0.000000,0.0,0.000000
217,0.206879,0.000000,0.000000,0.005085,0.009752,0.002849,0.000000,0.040357,0.000000,0.000000,0.056353,0.016781,0.0,0.000000,0.0,0.007033
470,0.203356,0.000000,0.005755,0.010980,0.016846,0.009353,0.000000,0.000000,0.007240,0.004912,0.000463,0.000000,0.0,0.000000,0.0,0.000000
730,0.202145,0.001735,0.045036,0.000000,0.015897,0.008130,0.000000,0.000000,0.018570,0.000000,0.000000,0.057581,0.0,0.030845,0.0,0.021509


In [13]:
test_vec = pd.DataFrame([
    [0.8, 0.2],
    [0.6, 0.4],
    [0.0, 0.7],
    [0.8, 0.4],
    [0.6, 0.8],
    [0.6, 0.1]
    ])

check_vec = np.array([1.0, 0.0])

In [21]:
dists = find_comps(test_vec, check_vec)

[5, 0, 3, 1, 4]
('Alameda', 'portland', '8_art')
('Adams Morgan', 'washingtondc', '10_attractions')
('Adams-Normandie', 'losangeles', '3_car')
('Adams North', 'sandiego', '1_taco')
('Admiral', 'seattle', '14_water')
7.0
8_art


In [22]:
dists

{0: 0.029857499854668124,
 1: 0.16794970566215628,
 2: 1.0,
 3: 0.10557280900008414,
 4: 0.40000000000000002,
 5: 0.013606076167856251}

In [23]:
sorted(dists.items(), key=operator.itemgetter(1))

[(5, 0.013606076167856251),
 (0, 0.029857499854668124),
 (3, 0.10557280900008414),
 (1, 0.16794970566215628),
 (4, 0.40000000000000002),
 (2, 1.0)]

In [24]:
sorted(dists.items(), key=operator.itemgetter(1))[0]

(5, 0.013606076167856251)

In [25]:
sorted(dists, key=dists.get)[:5]

[5, 0, 3, 1, 4]